In [1]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer , BertTokenizer, GPT2PreTrainedModel, GPT2Model, GPT2Config, GPT2LMHeadModel
import torch
from typing import Optional, Tuple
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from typing import*
from torch import tensor

PRETRAINED_MODEL_NAME = "gpt2"
Padding_id = 50256
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
def deal(tokenizer, file_path, data_size=0):
#處理好 dataframe
    df_train = pd.read_csv(file_path)
    if(data_size != 0):
        # df_train = df_train.sample(data_size)
        # df_train = df_train.reset_index(drop=True)
        df_train = df_train[0:data_size]
        df_train = df_train.sample(data_size).reset_index(drop=True)

    # question,context

    for i in range(len(df_train)):
        if( len(tokenizer.tokenize(df_train['question'][i])) + len(tokenizer.tokenize(df_train['context'][i])) >= 510 ):
            df_train = df_train.drop(i, axis=0)
            i = i-1
    
    df_train = df_train.reset_index(drop=True)
    
    df_train['answer_start'] = df_train['answer_start'].apply(lambda x : -1 if np.isnan(x) else int(x))
    df_train['text'] = df_train['text'].fillna(' ')
    df_train['text'] = df_train['text'].apply(lambda x : x.lower())
    df_train['label'] = df_train['answer_start'].apply(lambda x : 0 if x == -1 else 1)
    df_train = df_train[df_train.label == 1]
    df_train = df_train.drop(["label"], axis=1)

    # if ('index' in df_train.keys()):
    #     df_train = df_train.drop('index', axis=1)
    if ('c_id' in df_train.keys()):
        df_train = df_train.drop('c_id', axis=1)
    if 'Unnamed: 0' in df_train.keys():
        df_train = df_train.drop('Unnamed: 0', axis=1)
    
    return df_train

In [5]:
def my_pad_sequence(
    sequences: Union[tensor, List[tensor]],
    batch_first: bool = False,
    padding_value: int = 0,
    pos: str = 'right',
) -> tensor:

    if pos == 'right':
        padded_sequence = torch._C._nn.pad_sequence(sequences, batch_first, padding_value)
    elif pos == 'left':
        sequences = tuple(map(lambda s: s.flip(0), sequences))
        padded_sequence = torch._C._nn.pad_sequence(sequences, batch_first, padding_value)
        _seq_dim = padded_sequence.dim()
        padded_sequence = padded_sequence.flip(-_seq_dim+batch_first)
    else:
        raise ValueError("pos should be either 'right' or 'left', but got {}".format(pos))
    return padded_sequence


def QA_collate_batch(sample): #sample is List
    input_ids_batch = [s[0] for s in sample]
    label_batch = [s[1] for s in sample]

    input_ids_batch = my_pad_sequence(input_ids_batch, batch_first=True, padding_value = Padding_id, pos='left')
    label_batch = my_pad_sequence(label_batch, batch_first=True, padding_value = -100, pos='right')
    attn_batch = (input_ids_batch != Padding_id).int()

    return input_ids_batch, attn_batch, label_batch


class QAdataset(Dataset):
    def __init__(self, df, tokenizer):
        self.tokenizer = tokenizer
        self.df = df
    
    def __getitem__(self, index):
        df = self.df
        EC = self.tokenizer.encode_plus(df['context'][index], df['question'][index])
        input_ids = torch.tensor(EC['input_ids'], dtype=torch.long)
        
        ECl = self.tokenizer.encode_plus(df['text'][index] + '<|endoftext|>')
        ## ECl = self.tokenizer.encode_plus(df['text'][index])
        label = torch.tensor(ECl['input_ids'], dtype=torch.long)

        return input_ids, label
    
    def __len__(self):
        return len(self.df)

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer.pad_token_id = Padding_id

In [8]:
tokenizer.encode_plus('<|endoftext|>')
tokenizer.encode_plus('?')

{'input_ids': [30], 'attention_mask': [1]}

In [9]:
import pandas as pd 
from IPython.display import clear_output
from torch.utils.data import DataLoader
import random

Batch_size = 4
File_path = "/kaggle/input/asdfffff/SQuAD.csv"

df = deal(tokenizer,File_path,800) 
ds = QAdataset(df, tokenizer) 
dataloader = DataLoader(ds, batch_size=Batch_size, shuffle=True, collate_fn=QA_collate_batch)

In [16]:
ds[0]

(tensor([21993,   261, 32682,   338, 12435,  2837, 32727,  1440, 19318,  3080,
            13,   449,  1118, 15564, 11330,   607,  8216,   290,  4628,  4679,
           355,  3573, 18778,    11, 12059,   607,  3809,   355,   366,   505,
           286,   262,   749, 13206, 12834,   287,  2968,  2647,  1911,  2893,
          1194,  4014,  1139,   673,   318,   257,   366,    53,  4374,   936,
         40655,    11,   852,  1498,   284,  1702,   890,   290,  3716,  7758,
          1042,   292,   290, 12435,  4539, 42241,    11,   290,   287,  1994,
            13,  2332, 12435,  7883,  1612,   673,   318,  5174,   355,   262,
         44595,   286, 17886,   338,  5932,    13,   383,  6714, 11099,  3848,
         37361, 32682,   338,  3809,   366,   690, 12610,  1600,  6007,   286,
         13504,  1176,  2613,  5643,    11,  5848,    11,  3881,   894,   889,
            11,  1515,  1512, 10601,  5614,    11,   290, 10359,  1725,    13,
          5966,   350,   533,   829,   286,   383,  

In [11]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        model = GPT2LMHeadModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.transformer = model.transformer
        self.lm_head = model.lm_head
        self.config = self.transformer.config
        # self.lm_head = torch.nn.Linear(self.config.n_embd, self.config.vocab_size, bias=False)
    
    def forward(self,
                input_ids,
                attention_mask,
                past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
                encoder_hidden_states: Optional[torch.Tensor] = None,
                encoder_attention_mask: Optional[torch.FloatTensor] = None,
                labels: Optional[torch.LongTensor] = None,
                ) :

        if labels is not None :
            predict_logits = self.trainmode(input_ids=input_ids,
                                            attention_mask=attention_mask,
                                            labels=labels
                                            )
#             print(predict_logits)
            lossfn = torch.nn.CrossEntropyLoss()
            loss = lossfn(predict_logits.view(-1, predict_logits.size(-1)), labels.view(-1))

        return loss
    
    def trainmode(self,
                  input_ids,
                  attention_mask,
                  labels: Optional[torch.LongTensor] = None, ## batch, seqlen
                  ):
        padding_token_id = self.config.pad_token_id
        
        predict_logits = []
        batch_size = input_ids.size()[0]
        should_pred_len = labels.size()[1]
        curlen = 0

        unfinished_sentence = torch.tensor([1] * batch_size) ## batch_size
        ### when first run the sentence, there is no past_key_values
        ### so need the past_key_values for the setence, and then predict later
        
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)
        transformer_outputs = self.transformer(
            input_ids = input_ids,
            attention_mask = attention_mask
        )
        input_ids = input_ids.cpu()
        attention_mask = attention_mask.cpu()

        #### the three need return is (1)last_hidden (2)logits (3)past_key_values

        last_hidden = transformer_outputs.last_hidden_state ## batch size, seq_length, tensor_size
        logits = self.lm_head(last_hidden).cpu()  ## batch size, seq_length, vacabulary size
        last_hidden = last_hidden.cpu()
        past_key_values = transformer_outputs.past_key_values  ## layer, 2, batchsize, num_head, seq_len, one_head_len

        
        ##check print(labels)
        
        #### predict the label
        this_peer_finished = False
        while this_peer_finished == False:

            #### chose the predict logits from pre round #######################################################
            ####################################################################################################
            #### the logits_from_pre_step must not "all is eos"
            logits_from_pre_step = logits[:,-1,:]  ## batch, vacabulary size
            prob = torch.nn.functional.softmax(logits_from_pre_step, dim=-1)  ## batch, vocab_size
            selected_index = torch.argmax(prob, dim=-1)  ## batch
            # pred_tokens_id = prob[:, selected_index].unsqueeze(dim=-1)  ## batch , 1
            pred_tokens_id = selected_index.unsqueeze(dim=-1)  ## batch, 1 ~~ this is the new token id for each sequence
            predict_logits.append(logits_from_pre_step)
            ##############################################################################################
            

            #### prepare the new input, and the output for next round(step) ####################################
            ####################################################################################################
            #### random pick the next input
            rnd = torch.tensor([random.random() for _ in range(batch_size)])
            pick_answer = (rnd > 0.2).int()
            pick_answer = pick_answer * unfinished_sentence
            ### if has finished the sentence, then the label[k,cur_len] should be the -100, so cannot choose it
            pick_answer = pick_answer.unsqueeze(dim=-1)
            input_ids = (1-pick_answer) * pred_tokens_id + pick_answer * labels[:,curlen].unsqueeze(dim=-1)
            
#             print(pred_tokens_id)
#             print(input_ids)
            
            
            ##check print(pick_answer)
            ##check print(input_ids)
            
            # input_ids = pred_tokens_id
            #### 

            #### get new attention mask
            
            ##### all set to attention
            #### new_attention_mask = torch.tensor([[1]] * batch_size, dtype=torch.int32)
            
            #### after one eof appear in prediction, only attented the first eof, other is mask
            new_attention_mask = unfinished_sentence[:,None]
            # print(new_attention_mask)
            
            attention_mask = torch.cat([attention_mask, new_attention_mask], dim=-1) 
            ####
            
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            
            outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, past_key_values=past_key_values)
            
            input_ids = input_ids.cpu()
            attention_mask = attention_mask.cpu()
            ####################################################################################################

            #### renew the past key value, logits ##############################################################
            ####################################################################################################
            next_last_hidden = outputs.last_hidden_state  ## (batch, 1, hidden_size)
            logits = self.lm_head(next_last_hidden).cpu() ## (batch, 1, vocab_size)
            last_hidden = torch.cat([last_hidden, next_last_hidden.cpu()], dim=1)
            past_key_values = outputs.past_key_values
            #### 
            ####################################################################################################

            
            #### if the string has finished(mean it has appeared eos before), then set this round output to eos
            pred_tokens_id = pred_tokens_id * unfinished_sentence.unsqueeze(-1) + padding_token_id * (1-unfinished_sentence.unsqueeze(-1))
            #### the next round 

            #### renew the unfinished_sentence
            #### if all the output of this round is eos, set the unfinished_sentence[i] to 1
            #### if the label[k,curlen] is [pad], set it to finished, or there wiil be label id -100 to the input ids
            unfinished_sentence = unfinished_sentence * pred_tokens_id.ne(torch.tensor([padding_token_id]) ).squeeze()
            unfinished_sentence = unfinished_sentence * labels[:,curlen].ne(torch.tensor([padding_token_id]) )
            # if unfinished_sentence.max() == 0:
            #     this_peer_finished = True
    
            curlen += 1
            if curlen == should_pred_len:
                this_peer_finished = True

        ##############################################
        predict_logits = torch.stack(predict_logits)
        predict_logits = predict_logits.reshape(batch_size, labels.size()[1], -1) ## seq_len, batch, vocab_size -> batch, seq_len, vocab_size
        return predict_logits
            

In [7]:
for i in dataloader :
    print(i)
    break

(tensor([[ 2202,  3035,   604,  ...,   262,  4845,    30],
        [50256, 50256, 50256,  ...,  2613,  3823,    30],
        [50256, 50256, 50256,  ...,   636,   286,    30],
        [50256, 50256, 50256,  ...,  6157,   351,    30]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[  259,   257,  2008, 40689,   496, 50256],
        [ 9804, 50256,  -100,  -100,  -100,  -100],
        [15219,   338,  1259,  1326, 50256,  -100],
        [16520,  3541,   338,  1200, 50256,  -100]]))


In [12]:
model = MyModel()
# model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(DEVICE)
model.config.pad_token_id = Padding_id

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, nesterov = True)

for data in dataloader:
    input_id, attention_mask , label = data
    
#     input_id = input_id.to(DEVICE)
#     attention_mask = attention_mask.to(DEVICE)
#     label = label.to(DEVICE)
    
    loss = model(input_ids = input_id, attention_mask = attention_mask, labels = label)
    print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


tensor([[ 30],
        [ 30],
        [ 30],
        [383]])
tensor([[   30],
        [13330],
        [ 1845],
        [39123]])
tensor([[  30],
        [  30],
        [  30],
        [1440]])
tensor([[-100],
        [ 731],
        [ 271],
        [1440]])


/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [1,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [1,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [1,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [1,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [1,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [1,0,0], thread: [5,0,0] Assertion `srcIndex < srcSel

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model